<a href="https://colab.research.google.com/github/REEENG9/ML_2022/blob/main/Final_Exam_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Exam Lap Session

**RULES**
* **TODO** 1~4 번에 해당하는 모델링 부분만 작성하고, 나머지 코드는 일체 수정하지 않는 것을 원칙으로 함.
  * 수정 시 감점 요소가 될 수 있으며, 부득이하게 수정 시 수정 사항을 주석으로 필히 명시해야함. 
  * `utils.py`, `train.py`은 절대 수정 불가.
* 완성된 `.ipynb`을 다운 받아서 이루리에 제출해야함.
* 인터넷을 참조하는 것은 얼마든지 가능하나 수강생 간 코드 공유는 학칙에 따라 `F` 부여.

---

* In principle, only the modeling parts corresponding to **TODO** 1 to 4 can be modified, and the rest of the code should not be modified.
   * Points may be deducted if modify other codes, and modification must be clearly stated in comments when unavoidable corrections are made.
   * Never modified `utils.py` and `train.py`.
* You must download the completed `.ipynb` and submit it to `E-RURI`.
* It is possible to refer any resources in Internet, but the code sharing among students will be given an ‘F’ according to the school regulations.


# Preparation

In [1]:
!git clone https://github.com/choi-hyunsoo/machine_learnig_final.git

Cloning into 'machine_learnig_final'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 29 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), done.


In [2]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt

In [3]:
from machine_learnig_final.utils import *
from machine_learnig_final.train import *

In [4]:
# set random seed
set_seed()

# load data
train_iterator, valid_iterator, test_iterator = load_MNIST_data()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting .data/MNIST/raw/train-images-idx3-ubyte.gz to .data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting .data/MNIST/raw/train-labels-idx1-ubyte.gz to .data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting .data/MNIST/raw/t10k-images-idx3-ubyte.gz to .data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting .data/MNIST/raw/t10k-labels-idx1-ubyte.gz to .data/MNIST/raw



In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

# TODO
본 시험은 `MNIST` 데이터를 이용한 classifcation 문제를 다양한 Neural Networks 모델들로 구현하고 그 결과를 비교하는 것을 목표로 한다. 각 모델에 대한 그림과 설명을 참조하여 각 모델의 class를 완성하시오.

---

This exam aims to implement the classifcation problem using `MNIST` data with various Neural Networks models and compare the results. Refer to the picture and description for each model and complete the class of each model.

### 1. Multi-layer Perceptron [pts. 20]
아래와 같은 구조의 MLP를 구축하시오.
각 layer 사이에는 `ReLu` activation을 사용하시오.

<div>
<img src="https://github.com/choi-hyunsoo/machine_learnig_final/blob/main/mlp.png?raw=true" align="center" width="50%"/>
</div>

In [6]:
class MLP(nn.Module):
    def __init__(self, output_dim, input_dim=32*32*1):
        super().__init__()
        self.output_dim = output_dim

        #### complete the code ####
        self.hidden_fc = nn.Linear(input_dim, 50)
        self.output_fc = nn.Linear(50, output_dim)
        ###########################

    def forward(self, x):   
        #### complete the code ####
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)

        s = self.hidden_fc(x)
        z = F.sigmoid(s)

        out = self.output_fc(z)
        ###########################
        return out

### 2. Convolutional Neural Networks [pts. 20]
아래 그림과 같은 CNN 모델을 완성하시오.
Fully connectec Layer 직전에 `nn.Dropout(0.5)`를 추가하시오.

(힌트, `Lab5-2`를 참조하시오.)

---

Bulild the CNN model as shown in the figure below.
Add `nn.Dropout(0.5)` just before the Fully connectec Layer.

(Hint, refer to `Lab5-2`.)

---

<div>
<img src="https://github.com/choi-hyunsoo/machine_learnig_final/blob/main/cnn.png?raw=true" align="center" width="80%"/>
</div>

In [29]:
class CNN(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.output_dim = output_dim

        #### complete the code ####
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, 3, 2, 1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 192, 3, 1, 1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 384, 3, 1, 1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(384*2*2 , output_dim)
        )
        ###########################

    def forward(self, x):
        #### complete the code ####
        x = self.features(x)
        h = x.view(x.shape[0], -1)
        out = self.classifier(h)
        ###########################
        return out

### 3. Convolutional Neural Networks with Skip Connection [pts. 20]
Deep architectures에서 skip connection은 특정 layer의 output을 몇 개의 layer를 건너뛰고 다음 layer의 input에 추가하는 것이다.

본 모델에서는 input data를 flatten하여 1d vector로 만든 뒤, CNN의 feature와 concate ([`torch.cat`](https://pytorch.org/docs/stable/generated/torch.cat.html))하여 최종적으로 classification을 진행한다.

**앞선 2번 모델에 Skip Connection만 추가하는 것을 목표로 한다.**

---

In deep architectures, skip connection is where the output of a specific layer is skipped several layers and added to the input of the next layer.

In this model, after flattening the input data to make a 1d vector, concatenating it with the CNN feature ([`torch.cat`](https://pytorch.org/docs/stable/generated/torch.cat.html)). 

**Add only Skip Connection to the previous model No. 2**

<div>
<img src="https://github.com/choi-hyunsoo/machine_learnig_final/blob/main/cnn_SC.png?raw=true" align="center" width="80%"/>
</div>

In [33]:
class CNNwithSC(nn.Module):
    def __init__(self, output_dim, input_dim = 32 * 32 * 1):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim

        #### complete the code ####
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, 3, 2, 1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 192, 3, 1, 1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 384, 3, 1, 1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(384*2*2, output_dim)
        )
        ###########################

    def forward(self, x):
        #### complete the code ####
        x = self.features(x)
        h = x.view(x.shape[0], -1)
        out = self.classifier(torch.cat(h, dim=0))
        ###########################
        return out

### 4. Recurrent Neural Networks Model [pts. 20]
아래 그림과 같은 RNN 모델을 완성하시오.

(힌트, input data의 dimension의 문제가 발생할 경우 `.squeeze()` method를 사용하여 해결 할 수 있음.)

---

Complete the RNN model as shown in the figure below.

(Hint, if there is a problem with the dimension of input data, it can be solved by using the `.squeeze()` method.)

---

<div>
<img src="https://github.com/choi-hyunsoo/machine_learnig_final/blob/main/rnn.png?raw=true" align="center" width="50%"/>
</div>

In [34]:
class RNN(nn.Module):    
    def __init__(self, output_dim, input_dim=32,
                 num_layers=3, hidden_dim=512):
        super(RNN, self).__init__()

        #### complete the code ####
        self.lstm = nn.LSTM(input_dim, output_dim, num_layers)

        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sig = nn.Sigmoid()
        ###########################
    
    def forward(self, x):
        #### complete the code ####
        batch_size = x.size(0)
        lstm_out, hidden = self.lstm(x)
        fcout = self.fc(lstm_out[:, -1, :])
        out = self.sig(fcout)
        ###########################
        return out       

### Train the defined model and evalute [**no need to modify**]

In [35]:
for i, arch in enumerate([MLP, CNN, CNNwithSC, RNN]):
  try:
    print(f'----------------------{i}th model----------------------------')
    model = arch(output_dim=10).apply(initialize_parameters)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    model = model.to(device)
    print(f'The model has {count_parameters(model):,} trainable parameters')

    model = train_all(model, train_iterator, valid_iterator, optimizer, criterion, device)

    test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
    print(f'-----------------------------------------------------------\n')

  except:
    print(f'Something wrong with {i}th model')    
    print(f'-----------------------------------------------------------\n')

----------------------0th model----------------------------
The model has 51,760 trainable parameters


Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

Training:   0%|          | 0/211 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]

...

Training:   0%|          | 0/211 [00:00<?, ?it/s]

Something wrong with 2th model
-----------------------------------------------------------

----------------------3th model----------------------------
The model has 8,650 trainable parameters


Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

Training:   0%|          | 0/211 [00:00<?, ?it/s]

Something wrong with 3th model
-----------------------------------------------------------

